https://mlflow.org/docs/latest/python_api/mlflow.html

In [ ]:
pip install mlflow

In [ ]:
import mlflow

Отдельный объект создавать не нужно, просто вызываем необходимые методы из библиотеки

In [ ]:
with mlflow.start_run(run_name='MLFlow_experiment'):
    mlflow.log_artifact("mlflow_exmpl.py")
    run_experiment()

В папке проекта, где лежит запускаемый код, создаться папка `mlruns` и там будут лежать папки, которые будут хранить все логи по всем запускам

В терминале, будучи в папке, где лежит папка `mlruns` запускаем в консоли 

`mlflow ui`

<img src="./images/mlflow.png" alt="Drawing" />

# Пример

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')

import torch

print(torch.__version__)

import torch.nn as nn # содержит функции для реалзации архитектуры нейронных сетей
import torch.optim as optim
import torch.utils.data as data_utils

from pytorch_lightning.metrics import Accuracy

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

import mlflow

1.5.0


In [2]:
INPUT_SIZE = 37
HIDDEN_SIZE = 25
OUTPUT_SIZE = 4
LEARNING_RATE = 1e-2
EPOCHS = 400
BATCH_SIZE = 256

## ETL

In [3]:
def load_dataset():
    X = pd.read_csv('./data/X_cat.csv', sep='\t', index_col=0)
    target = pd.read_csv('./data/y_cat.csv', sep='\t', index_col=0, names=['status'])  # header=-1,

    print(X.shape)
    print(X.head())

    target = target.iloc[:, :].values
    target[target == 'Died'] = 'Euthanasia'

    le = LabelEncoder()
    y = le.fit_transform(target)

    return X, y

In [4]:
def create_data_loader(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X.values, y,
                                                        test_size=0.2, stratify=y, random_state=42)
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    train_tensor = data_utils.TensorDataset(torch.tensor(X_train.astype(np.float32)), torch.tensor(y_train))
    train_loader = data_utils.DataLoader(dataset=train_tensor,
                                         batch_size=BATCH_SIZE,
                                         shuffle=True)

    test_tensor = data_utils.TensorDataset(torch.tensor(X_test.astype(np.float32)), torch.tensor(y_test))
    test_loader = data_utils.DataLoader(dataset=test_tensor,
                                        batch_size=BATCH_SIZE,
                                        shuffle=False)
    
    return X_train, X_test, y_train, y_test, train_loader, test_loader

## Model Architecture

In [5]:
class MLPNet(nn.Module):

    def __init__(self, input_size, hidden_size, output_size):
        super(MLPNet, self).__init__()

        self.linear1 = torch.nn.Linear(input_size, hidden_size)

        self.linear2 = torch.nn.Linear(hidden_size, hidden_size)

        self.linear3 = torch.nn.Linear(hidden_size, output_size)

    def forward(self, x):
        output = self.linear1(x)
        output = torch.relu(output)

        output = self.linear2(output)
        output = torch.relu(output)

        output = self.linear3(output)
        predictions = torch.softmax(output, dim=1)

        return predictions

## Train pipeline

In [6]:
def run_train(train_loader, test_loader):
    model = MLPNet(INPUT_SIZE, HIDDEN_SIZE, OUTPUT_SIZE)

    criterion = nn.CrossEntropyLoss()
    accuracy = Accuracy()
    optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE)

    step = 0
    for epoch in range(EPOCHS):
        model.train()

        for features, label in train_loader:
            # Reset gradients
            optimizer.zero_grad()

            output = model(features)
            # Calculate error and backpropagate
            loss = criterion(output, label)
            loss.backward()
            acc = accuracy(output, label).item()

            # Update weights with gradients
            optimizer.step()

            mlflow.log_metric('Train/CrossEntropyLoss', loss.item(), step)
            mlflow.log_metric('Train/Accuracy', acc, step)

            step += 1

            if step % 50 == 0:
                print('EPOCH %d STEP %d : train_loss: %f train_acc: %f' %
                      (epoch, step, loss, acc))


        # Run validation
        running_loss = []
        valid_scores = []
        valid_labels = []
        model.eval()
        with torch.no_grad():
            for features, label in test_loader:
                output = model(features)
                # Calculate error and backpropagate
                loss = criterion(output, label)

                running_loss.append(loss.item())
                valid_scores.extend(torch.argmax(output, dim=1))
                valid_labels.extend(label)

        valid_accuracy = accuracy(torch.tensor(valid_scores), torch.tensor(valid_labels)).item()

        mlflow.log_metric('Valid/CrossEntropyLoss', np.mean(running_loss), step)
        mlflow.log_metric('Valid/Accuracy', valid_accuracy, step)

        print('EPOCH %d : valid_loss: %f valid_acc: %f' % (epoch, np.mean(running_loss), valid_accuracy))

    return

In [7]:
def run_experiment():
    features, labels = load_dataset()
    X_train, X_test, y_train, y_test, train_loader, test_loader = create_data_loader(features, labels)

    mlflow.log_param("LEARNING_RATE", LEARNING_RATE)
    mlflow.log_metric("INPUT_SIZE", INPUT_SIZE)
    mlflow.log_metric("HIDDEN_SIZE", HIDDEN_SIZE)
    mlflow.log_param("NROF_CLASSES", OUTPUT_SIZE)
    mlflow.log_metric("BATCH_SIZE", BATCH_SIZE)

    run_train(train_loader, test_loader)

    return

In [8]:
with mlflow.start_run(run_name='MLFlow_experiment'):
    mlflow.log_artifact("mlflow_exmpl.py")
    run_experiment()

(26729, 37)
   IsDog    Age  HasName  NameLength  NameFreq  MixColor  ColorFreqAsIs  \
0      1  365.0        1           7  0.000157         1       0.032919   
1      0  365.0        1           5  0.000655         0       0.008092   
2      1  730.0        1           6  0.000052         1       0.026293   
3      0   21.0        0           7  0.285871         0       0.000471   
4      1  730.0        0           7  0.285871         0       0.023831   

   ColorFreqBase  TabbyColor  MixBreed  ...  SexStatus_Flawed  \
0       0.463624           0         1  ...                 1   
1       0.015005           1         1  ...                 1   
2       0.357521           0         1  ...                 1   
3       0.058418           0         1  ...                 0   
4       0.075353           0         0  ...                 1   

   SexStatus_Intact  SexStatus_Unknown  Weekday_0  Weekday_1  Weekday_2  \
0                 0                  0          0          0          1

EPOCH 42 : valid_loss: 1.134491 valid_acc: 0.606435
EPOCH 43 STEP 3650 : train_loss: 1.127534 train_acc: 0.613281
EPOCH 43 : valid_loss: 1.134093 valid_acc: 0.606248
EPOCH 44 STEP 3700 : train_loss: 1.131543 train_acc: 0.617188
EPOCH 44 STEP 3750 : train_loss: 1.126128 train_acc: 0.609375
EPOCH 44 : valid_loss: 1.133719 valid_acc: 0.606248
EPOCH 45 STEP 3800 : train_loss: 1.094194 train_acc: 0.640625
EPOCH 45 STEP 3850 : train_loss: 1.134531 train_acc: 0.605469
EPOCH 45 : valid_loss: 1.133369 valid_acc: 0.606435
EPOCH 46 STEP 3900 : train_loss: 1.108854 train_acc: 0.636719
EPOCH 46 : valid_loss: 1.133036 valid_acc: 0.606622
EPOCH 47 STEP 3950 : train_loss: 1.085194 train_acc: 0.648438
EPOCH 47 STEP 4000 : train_loss: 1.162586 train_acc: 0.578125
EPOCH 47 : valid_loss: 1.132722 valid_acc: 0.606622
EPOCH 48 STEP 4050 : train_loss: 1.133998 train_acc: 0.605469
EPOCH 48 STEP 4100 : train_loss: 1.110153 train_acc: 0.625000
EPOCH 48 : valid_loss: 1.132424 valid_acc: 0.606622
EPOCH 49 STEP 41

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/firiuza/anaconda2/envs/pytorch_env/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-15ad048495d0>", line 3, in <module>
    run_experiment()
  File "<ipython-input-7-86706572fcfe>", line 11, in run_experiment
    run_train(train_loader, test_loader)
  File "<ipython-input-6-8e219dea6eae>", line 19, in run_train
    loss.backward()
  File "/home/firiuza/anaconda2/envs/pytorch_env/lib/python3.6/site-packages/torch/tensor.py", line 198, in backward
    torch.autograd.backward(self, gradient, retain_graph, create_graph)
  File "/home/firiuza/anaconda2/envs/pytorch_env/lib/python3.6/site-packages/torch/autograd/__init__.py", line 100, in backward
    allow_unreachable=True)  # allow_unreachable flag
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "

KeyboardInterrupt: 